In [3]:
import face_recognition
import cv2
import os
import os.path
import pickle
import matplotlib.pyplot as plt
import numpy as np
from imutils import paths

In [ ]:
def detectFace(img_path):
    image = cv2.imread(img_path)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    print("[INFO] recognizing faces...")
    boxes = face_recognition.face_locations(rgb,model="cnn")
    encodings = face_recognition.face_encodings(rgb, boxes)
    names = []
    for encoding in encodings:
        print(encoding)
        matches = face_recognition.compare_faces(data["encodings"],encoding)
        name = "Unknown"
        if True in matches:
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            counts = {}
            for i in matchedIdxs:
                name = data["names"][i]
                counts[name] = counts.get(name, 0) + 1
            name = max(counts, key=counts.get)
        names.append(name)
    
    # loop over the recognized faces
    for ((top, right, bottom, left), name) in zip(boxes, names):
        # draw the predicted face name on the image
        cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
        y = top - 15 if top - 15 > 15 else top + 15
        cv2.putText(image, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX,
            0.75, (0, 255, 0), 2)

    # show the output image
    cv2.imshow("Image", image)
    cv2.waitKey(0)
    
# detectFace("./input/img1.jpg")

In [57]:
def recogizeFace(target_face_encoding):
#     target_face_encoding = face_recognition.face_encodings(target_face)
    # load the known faces and embeddings
    print("[INFO] loading encodings...")
    data = pickle.loads(open("encodings.pickle", "rb").read())
    # loop over the facial embeddings
    name = "Unknown"
    matches = face_recognition.compare_faces(data["encodings"], target_face_encoding)
    # check to see if we have found a match
    if True in matches:
        matchedIdxs = [i for (i, b) in enumerate(matches) if b]
        counts = {}
        for i in matchedIdxs:
            name = data["names"][i]
            counts[name] = counts.get(name, 0) + 1
        name = max(counts, key=counts.get)

    return name
    

In [6]:
import face_recognition
import cv2
import os
import os.path
import pickle
import matplotlib.pyplot as plt
import numpy as np
from imutils import paths

def trainModel1():
    path = './dataset'
    knownEncodings = []
    knownNames = []
    # r=root, d=directories, f = files
    index = 0
    print("start load dataset to training")
    for r, d, f in os.walk(path):
        for folder in d:
            name = folder
            for r1, d1, f1 in os.walk(os.path.join(r, folder)):
                for file in f1:
                    print(os.path.join(r, folder, file))
                    image_loaded = face_recognition.load_image_file(os.path.join(r, folder, file))
                    face_encodings = face_recognition.face_encodings(image_loaded)
                    # loop over the encodings
                    for encoding in face_encodings:
                        # add each encoding + name to our set of known names and
                        # encodings
                        knownEncodings.append(encoding)
                        knownNames.append(name)
    print("encoding done")
    print("start write data to file")
    print("[INFO] serializing encodings...")
    data = {"encodings": knownEncodings, "names": knownNames}
    f = open("encodings.pickle", "wb")
    f.write(pickle.dumps(data))
    f.close()
    print("training done")
    
trainModel1()

start load dataset to training
./dataset/TRUONGTX/IMG_5328_face002.jpg
./dataset/TAIHPT/face_19.jpg
./dataset/DUYNN/duynn01.jpg
./dataset/DUYNN/duynn02.jpg
./dataset/TANTD/image_2.jpg
./dataset/TANTD/image_3.jpg
./dataset/TANTD/image_1.jpg
./dataset/TANTD/image_4.jpg
./dataset/THAODNT/face_52.jpg
./dataset/ThuyLTN/ThuyLTN.png
./dataset/MINHHA/face_51.jpg
./dataset/MINHHA/face_42.jpg
./dataset/MINHHA/face_4.jpg
./dataset/MINHHA/face_58.jpg
./dataset/DUYETLV/face-5.jpg
./dataset/DUYETLV/face-43.jpg
./dataset/DUYETLV/Screen Shot 2019-02-27 at 6.39.22 PM.png
./dataset/DUYETLV/Screen Shot 2019-02-27 at 6.39.12 PM.png
./dataset/DUYETLV/Screen Shot 2019-02-27 at 6.38.53 PM.png
./dataset/TANPV/TANPV-2.jpg
./dataset/TANPV/TANPV-1.jpg
./dataset/LOCTH/locth-01.jpg
./dataset/LOCTH/locth-03.jpg
./dataset/LOCTH/locth-02.jpg
./dataset/VUND/vund2.png
./dataset/VUND/vund3.png
./dataset/ANHLT/ANHLT-1.jpg
./dataset/ANHLT/ANHLT-2.jpg
./dataset/NHANTH/face_50.jpg
./dataset/MYNH/face_77.jpg
./dataset/VULQ/v

In [5]:
import face_recognition
import cv2
import os
import os.path
import pickle
import matplotlib.pyplot as plt
import numpy as np
from imutils import paths

def trainModel():
    # grab the paths to the input images in our dataset
    print("[INFO] quantifying faces...")
    imagePaths = list(paths.list_images("./dataset"))

    # initialize the list of known encodings and known names
    knownEncodings = []
    knownNames = []
    # loop over the image paths
    for (i, imagePath) in enumerate(imagePaths):
        # extract the person name from the image path
        print("[INFO] processing image {}/{}".format(i + 1, len(imagePaths)))
        name = imagePath.split(os.path.sep)[-2]
        print(name)
        # load the input image and convert it from BGR (OpenCV ordering)
        # to dlib ordering (RGB)
        image = cv2.imread(imagePath)
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # detect the (x, y)-coordinates of the bounding boxes
        # corresponding to each face in the input image
        boxes = face_recognition.face_locations(rgb, model="cnn")

        # compute the facial embedding for the face
        encodings = face_recognition.face_encodings(rgb, boxes)

        # loop over the encodings
        for encoding in encodings:
            # add each encoding + name to our set of known names and
            # encodings
            knownEncodings.append(encoding)
            knownNames.append(name)
            
    # dump the facial encodings + names to disk
    print("[INFO] serializing encodings...")
    data = {"encodings": knownEncodings, "names": knownNames}
    f = open("encodings.pickle", "wb")
    f.write(pickle.dumps(data))
    f.close()
    
#trainModel()

[INFO] quantifying faces...
[INFO] processing image 1/56
TRUONGTX
[INFO] processing image 2/56
TAIHPT
[INFO] processing image 3/56
DUYNN
[INFO] processing image 4/56
DUYNN
[INFO] processing image 5/56
TANTD
[INFO] processing image 6/56
TANTD
[INFO] processing image 7/56
TANTD
[INFO] processing image 8/56
TANTD
[INFO] processing image 9/56
THAODNT
[INFO] processing image 10/56
ThuyLTN
[INFO] processing image 11/56
MINHHA
[INFO] processing image 12/56
MINHHA
[INFO] processing image 13/56
MINHHA
[INFO] processing image 14/56
MINHHA
[INFO] processing image 15/56
DUYETLV
[INFO] processing image 16/56
DUYETLV
[INFO] processing image 17/56
DUYETLV


error: OpenCV(3.4.3) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/color.cpp:181: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
